# Flappy Bird AI на Q-Learning

В этой тетради мы создаём полностью автоматизированного агента, который учится играть в Flappy Bird при помощи Q-Learning и нейросети Linear QNet. Для наглядности ноутбук разбит на логические секции, а каждая строка кода снабжена подробными комментариями.

**Структура ноутбука:**
1. Импорт библиотек и подготовка среды выполнения.
2. Конфигурация игры и вспомогательные структуры данных.
3. Реализация Flappy Bird на pygame (физика, препятствия, визуализация и состояние).
4. Нейронная сеть Linear QNet, тренер Q-Learning и визуализация метрик.
5. Агент с памятью опыта, epsilon-greedy стратегией и функцией обучения.
6. Запуск цикла обучения на 200 играх, сохранение лучшей модели и построение графиков.
7. Генерация GIF с игрой обученного ИИ через create_ai_gif.

Все пояснения, описания параметров и комментарии приведены на русском языке и соответствуют требованиям задания.


## Коротко о Q-Learning и гиперпараметрах

Обновление оценки качества действия выполняется по формуле:

**Q_new = Q_old + alpha * (reward + gamma * max(Q_next) - Q_old)**

Где:
- *state* и *next_state* — векторы из 11 признаков (опасности, направление, цель).
- *action* — одно из трёх действий (ничего, взмах, пикирование).
- *alpha (learning rate)* — скорость обучения оптимизатора Adam.
- *gamma* — коэффициент дисконтирования будущих наград.

Linear QNet получает 11 входов, имеет скрытый слой на 256 нейронов и три выхода. Память опыта на 100 000 переходов реализует Experience Replay, а epsilon-greedy стратегия постепенно снижает долю случайных действий от 200 до 5 по мере прохождения 200 игр.


In [ ]:

import os  # Импортируем модуль os для управления системными переменными окружения
os.environ["SDL_AUDIODRIVER"] = "dummy"  # Отключаем аудиодрайвер pygame, чтобы запуск проходил без звука
os.environ["SDL_VIDEODRIVER"] = "dummy"  # Включаем виртуальный видеодрайвер, чтобы окно pygame работало без физического дисплея
import random  # Используем random для стохастики среды и стратегии epsilon-greedy
from collections import deque  # deque обеспечивает быструю реализацию памяти опыта с ограниченным размером
from dataclasses import dataclass  # dataclass используется для компактного описания конфигураций и структур данных
from typing import Deque, List, Tuple  # Импортируем типы для повышения читаемости и статической проверки
import numpy as np  # NumPy нужен для работы с векторами состояний и преобразованиями
import pygame  # pygame отвечает за визуализацию и игровую механику Flappy Bird
import torch  # PyTorch используется для реализации нейросети агента
import torch.nn as nn  # Подмодуль nn содержит готовые слои и функции активации
import torch.optim as optim  # Подмодуль optim предоставляет оптимизаторы, включая Adam
from IPython import display as ipy_display  # display позволяет обновлять графики внутри Jupyter Notebook
import matplotlib.pyplot as plt  # Matplotlib используем для построения кривых очков и среднего значения
import imageio  # ImageIO понадобится для сохранения последовательности кадров в GIF
plt.style.use("seaborn-v0_8")  # Задаём стиль графиков, чтобы визуализация была читабельной с первого запуска
plt.ion()  # Включаем интерактивный режим Matplotlib для динамического обновления графиков
pygame.init()  # Инициализируем основной модуль pygame сразу после настройки драйверов
pygame.font.init()  # Отдельно активируем подсистему шрифтов для отображения текста на экране
random.seed(42)  # Фиксируем сид генератора случайных чисел Python для воспроизводимости
np.random.seed(42)  # Повторяем то же самое для NumPy, чтобы состояние среды было детерминировано
torch.manual_seed(42)  # Аналогично инициализируем сид PyTorch для повторяемого обучения


In [ ]:

@dataclass  # Используем dataclass, чтобы удобно группировать все параметры игры и обучения
class GameConfig:  # Класс GameConfig хранит значения, контролирующие физику, визуализацию и обучение
    screen_width: int = 288  # Ширина окна Flappy Bird в пикселях
    screen_height: int = 512  # Высота окна игры
    fps: int = 60  # Количество кадров в секунду
    gravity: float = 0.5  # Постоянная гравитации, которая притягивает птицу вниз каждый кадр
    flap_strength: float = 9.5  # Скорость, которую получает птица при взмахе крыльями
    fast_drop_force: float = 3.0  # Дополнительное ускорение вниз при действии «пикирование»
    max_fall_speed: float = 12.0  # Ограничение на максимальную скорость падения
    pipe_gap: int = 140  # Высота отверстия между верхней и нижней трубой
    pipe_width: int = 52  # Ширина одной трубы
    pipe_distance: int = 200  # Минимальное расстояние между соседними трубами по оси X
    pipe_speed: int = 3  # Скорость сдвига труб влево каждый кадр
    bird_x: int = 50  # Горизонтальная позиция птицы
    bird_width: int = 34  # Ширина прямоугольника птицы
    bird_height: int = 24  # Высота прямоугольника птицы
    background_color: Tuple[int, int, int] = (135, 206, 235)  # Цвет неба в формате RGB
    pipe_color: Tuple[int, int, int] = (34, 139, 34)  # Цвет труб (тёмно-зелёный)
    bird_color: Tuple[int, int, int] = (255, 255, 0)  # Цвет птицы (жёлтый прямоугольник)
    text_color: Tuple[int, int, int] = (0, 0, 0)  # Цвет текста интерфейса (чёрный)
    alive_reward: float = 0.1  # Малое положительное вознаграждение за каждый прожитый кадр
    pipe_reward: float = 10.0  # Награда за успешное прохождение очередной трубы
    collision_penalty: float = -10.0  # Штраф за столкновение с трубой или выход за границы
    max_memory: int = 100_000  # Объём памяти опыта для Experience Replay
    batch_size: int = 1000  # Размер батча для обучения из памяти опыта
    lr: float = 0.001  # Скорость обучения оптимизатора Adam
    gamma: float = 0.9  # Коэффициент дисконтирования будущих наград
    max_games: int = 200  # Количество игр, которые проведём во время тренировки
    exploration_start: float = 200.0  # Начальное значение epsilon для стратегии epsilon-greedy
    exploration_end: float = 5.0  # Минимальное значение epsilon, чтобы всегда оставлять немного исследования
    exploration_decay: float = 1.0  # Значение, на которое уменьшается epsilon после каждой игры
    best_model_path: str = "best_flappy_q_agent.pth"  # Путь для сохранения лучшей модели агента
    gif_path: str = "flappy_q_learning.gif"  # Путь для итогового GIF с игрой ИИ
    gif_max_steps: int = 600  # Количество кадров, записываемых в GIF, чтобы показать несколько проходов

@dataclass  # Определяем структуру для характеристик трубы
class Pipe:  # Класс Pipe хранит положение X и вертикальный центр отверстия
    x: int  # Текущая координата X трубы
    gap_y: int  # Вертикальный центр отверстия между верхней и нижней частью
    scored: bool = False  # Флаг, чтобы начислять очки ровно один раз за трубу

@dataclass  # Структура для элемента памяти опыта агента
class Experience:  # Мы будем сохранять состояние, действие, награду, новое состояние и признак завершения
    state: np.ndarray  # Состояние среды до действия (11 параметров)
    action: List[int]  # Вектор из 3 значений (one-hot), описывающий действие агента
    reward: float  # Вознаграждение, полученное после действия
    next_state: np.ndarray  # Состояние после совершения действия
    done: bool  # Флаг окончания игры после этого шага


In [ ]:

class FlappyBirdGame:  # Класс, который инкапсулирует всю игровую механику Flappy Bird
    def __init__(self, config: GameConfig):  # Конструктор принимает объект конфигурации
        self.config = config  # Сохраняем конфигурацию для дальнейшего использования
        self.screen = pygame.display.set_mode((config.screen_width, config.screen_height))  # Создаём окно pygame с указанными размерами
        pygame.display.set_caption("Flappy Bird Q-Learning")  # Подписываем окно, чтобы легче отслеживать симуляцию
        self.clock = pygame.time.Clock()  # Создаём объект Clock для контроля FPS
        self.font = pygame.font.SysFont("arial", 18)  # Загружаем шрифт для отображения текста
        self.reset()  # Инициализируем состояние игры вызовом reset

    def reset(self) -> None:  # Сбрасываем игру к начальному состоянию
        self.bird_y = self.config.screen_height // 2  # Помещаем птицу в вертикальный центр экрана
        self.bird_velocity = 0.0  # Обнуляем вертикальную скорость
        self.score = 0  # Сбрасываем счёт очков
        self.frame_iteration = 0  # Обнуляем счётчик кадров
        self.pipes: List[Pipe] = []  # Очищаем список труб
        for _ in range(2):  # Создаём две трубы заранее
            self.spawn_pipe(initial=True)  # Добавляем трубу с флагом initial

    def spawn_pipe(self, initial: bool = False) -> None:  # Метод создаёт новую трубу и добавляет её в список
        gap_margin = 80  # Защитные поля сверху и снизу, чтобы отверстие не смещалось в край
        min_center = gap_margin + self.config.pipe_gap // 2  # Минимальный допустимый центр отверстия
        max_center = self.config.screen_height - gap_margin - self.config.pipe_gap // 2  # Максимальный допустимый центр отверстия
        gap_y = random.randint(min_center, max_center)  # Случайно выбираем положение отверстия
        if initial and self.pipes:  # Если это начальная генерация и уже есть трубы
            x_position = self.pipes[-1].x + self.config.pipe_distance  # Размещаем трубу на фиксированном расстоянии от предыдущей
        elif initial:  # Если это самая первая труба
            x_position = self.config.screen_width + 100  # Отодвигаем её вправо, чтобы игрок успел среагировать
        else:  # В остальных случаях
            x_position = self.config.screen_width + 20  # Размещаем трубу сразу за правой границей экрана
        self.pipes.append(Pipe(x=x_position, gap_y=gap_y))  # Добавляем новую трубу в список препятствий

    def handle_action(self, action_index: int) -> None:  # Метод применяет выбранное агентом действие
        if action_index == 1:  # Действие «взмах» заставляет птицу резко подниматься
            self.bird_velocity = -self.config.flap_strength  # Устанавливаем отрицательную скорость
        elif action_index == 2:  # Действие «пикирование» ускоряет падение
            self.bird_velocity += self.config.fast_drop_force  # Увеличиваем скорость вниз
        else:  # Если выбран вариант «ничего не делать»
            pass  # Птица продолжит двигаться под влиянием гравитации

    def apply_physics(self) -> None:  # Метод обновляет скорость и положение птицы
        self.bird_velocity += self.config.gravity  # Добавляем ускорение свободного падения
        self.bird_velocity = max(-self.config.flap_strength, min(self.bird_velocity, self.config.max_fall_speed))  # Ограничиваем скорость
        self.bird_y += self.bird_velocity  # Обновляем вертикальную позицию птицы

    def move_pipes(self) -> None:  # Сдвигаем трубы влево
        for pipe in self.pipes:  # Проходим по всем трубам
            pipe.x -= self.config.pipe_speed  # Уменьшаем координату X
        if self.pipes and self.pipes[0].x + self.config.pipe_width < 0:  # Если самая левая труба вышла за экран
            self.pipes.pop(0)  # Удаляем её из списка
        if not self.pipes or self.pipes[-1].x < self.config.screen_width - self.config.pipe_distance:  # Если нужно добавить новую трубу
            self.spawn_pipe()  # Создаём новую трубу

    def is_collision(self) -> bool:  # Проверяем столкновения
        if self.bird_y <= 0 or self.bird_y + self.config.bird_height >= self.config.screen_height:  # Столкновение с границами
            return True  # Сразу сообщаем о проигрыше
        for pipe in self.pipes:  # Проверяем каждую трубу
            bird_right = self.config.bird_x + self.config.bird_width  # Правая граница птицы
            pipe_right = pipe.x + self.config.pipe_width  # Правая граница трубы
            if bird_right > pipe.x and self.config.bird_x < pipe_right:  # Если по X прямоугольники пересекаются
                gap_top = pipe.gap_y - self.config.pipe_gap // 2  # Верхняя граница отверстия
                gap_bottom = pipe.gap_y + self.config.pipe_gap // 2  # Нижняя граница отверстия
                if self.bird_y < gap_top or self.bird_y + self.config.bird_height > gap_bottom:  # Птица вне прохода
                    return True  # Столкновение с трубой
        return False  # Если все проверки пройдены, столкновения нет

    def get_next_pipe(self) -> Pipe:  # Возвращаем ближайшую впереди трубу
        for pipe in self.pipes:  # Идём по трубам слева направо
            if pipe.x + self.config.pipe_width >= self.config.bird_x:  # Находим первую трубу, которая ещё не пройдена
                return pipe  # Возвращаем найденную трубу
        return self.pipes[0]  # Если все трубы позади, возвращаем самую левую

    def get_state(self) -> np.ndarray:  # Формируем вектор признаков состояния
        pipe = self.get_next_pipe()  # Получаем ближайшую трубу
        gap_top = pipe.gap_y - self.config.pipe_gap // 2  # Верхняя точка отверстия
        gap_bottom = pipe.gap_y + self.config.pipe_gap // 2  # Нижняя точка отверстия
        danger_straight = int(  # Проверяем, опасна ли ближайшая труба на текущей траектории
            pipe.x <= self.config.bird_x + self.config.bird_width + 5  # Смотрим, находится ли труба достаточно близко по горизонтали
            and (self.bird_y < gap_top or self.bird_y + self.config.bird_height > gap_bottom)  # Проверяем, не выходит ли траектория за пределы зазора по высоте
        )  # Преобразуем логическое выражение в 0 или 1 для признака опасности прямо
        danger_right = int(self.bird_y <= 0)  # Опасность у потолка (аналог «справа»)
        danger_left = int(self.bird_y + self.config.bird_height >= self.config.screen_height)  # Опасность у земли (аналог «слева»)
        dir_up = int(self.bird_velocity < -0.1)  # Признак движения вверх
        dir_down = int(self.bird_velocity >= -0.1)  # Признак движения вниз
        dir_left = 0  # Горизонтального движения влево нет
        dir_right = 1  # Птица всегда движется вправо относительно труб
        bird_center_y = self.bird_y + self.config.bird_height / 2  # Центр птицы по вертикали
        target_up = int(bird_center_y > pipe.gap_y)  # Цель находится выше
        target_down = int(bird_center_y < pipe.gap_y)  # Цель находится ниже
        target_left = int(pipe.x + self.config.pipe_width < self.config.bird_x)  # Цель позади
        target_right = int(pipe.x + self.config.pipe_width >= self.config.bird_x)  # Цель впереди
        state = np.array(  # Собираем упорядоченный набор бинарных признаков состояния
            [  # Начинаем перечисление всех 11 признаков
                danger_straight,  # Опасность прямо
                danger_right,  # Опасность сверху
                danger_left,  # Опасность снизу
                dir_up,  # Направление вверх
                dir_down,  # Направление вниз
                dir_left,  # Направление влево
                dir_right,  # Направление вправо
                target_up,  # Цель выше
                target_down,  # Цель ниже
                target_left,  # Цель позади
                target_right,  # Цель впереди
            ],  # Завершаем список признаков состояния
            dtype=int,  # Храним признаки как целые числа для совместимости с PyTorch
        )  # Возвращаем итоговый вектор состояния
        return state  # Возвращаем состояние агенту

    def draw_pipes(self) -> None:  # Вспомогательный метод для отрисовки всех труб
        for pipe in self.pipes:  # Проходим по списку труб
            gap_top = pipe.gap_y - self.config.pipe_gap // 2  # Считаем верхнюю границу отверстия
            gap_bottom = pipe.gap_y + self.config.pipe_gap // 2  # Считаем нижнюю границу отверстия
            pygame.draw.rect(self.screen, self.config.pipe_color, pygame.Rect(pipe.x, 0, self.config.pipe_width, gap_top))  # Рисуем верхнюю часть трубы как прямоугольник
            pygame.draw.rect(self.screen, self.config.pipe_color, pygame.Rect(pipe.x, gap_bottom, self.config.pipe_width, self.config.screen_height - gap_bottom))  # Рисуем нижнюю часть трубы с учётом свободного пространства

    def draw_bird(self) -> None:  # Метод рисует птицу как прямоугольник
        pygame.draw.rect(self.screen, self.config.bird_color, pygame.Rect(self.config.bird_x, int(self.bird_y), self.config.bird_width, self.config.bird_height))  # Отрисовываем птицу как жёлтый прямоугольник

    def update_ui(self, action_index: int, game_number: int) -> None:  # Метод обновляет визуализацию и выводит текстовую информацию
        self.screen.fill(self.config.background_color)  # Закрашиваем фон выбранным цветом неба
        pygame.draw.rect(self.screen, self.config.bird_color, pygame.Rect(self.config.bird_x, int(self.bird_y), self.config.bird_width, self.config.bird_height))  # Рисуем птицу как жёлтый прямоугольник
        game_surface = self.font.render(f"Game: {game_number}", True, self.config.text_color)  # Номер игры
    def update_ui(self, action_index: int, game_number: int) -> None:  # Метод обновляет визуализацию и выводит текстовую информацию
        self.screen.fill(self.config.background_color)  # Закрашиваем фон выбранным цветом неба
        self.draw_pipes()  # Рисуем все трубы на текущем кадре
        self.draw_bird()  # Отображаем птицу поверх труб
        action_names = {0: "Держим", 1: "Взмах", 2: "Пикирование"}  # Словарь для текстового отображения действий
        score_surface = self.font.render(f"Score: {self.score}", True, self.config.text_color)  # Создаём поверхность с текущим счётом
        action_surface = self.font.render(f"Action: {action_names[action_index]}", True, self.config.text_color)  # Подготавливаем текст с действием
        game_surface = self.font.render(f"Game: {game_number}", True, self.config.text_color)  # Текст с номером игры
        self.screen.blit(score_surface, (10, 10))  # Показываем счёт в левом верхнем углу
        self.screen.blit(action_surface, (10, 30))  # Размещаем текущие действия под счётом
        self.screen.blit(game_surface, (10, 50))  # Отображаем номер игры для контроля прогресса
        pygame.display.flip()  # Переворачиваем буфер, чтобы показать обновлённый кадр

                pygame.quit()  # Корректно завершаем pygame
                raise SystemExit("Окно pygame закрыто пользователем")  # Останавливаем выполнение ноутбука
        action_index = int(np.argmax(action_vector))  # Конвертируем one-hot действие в индекс 0/1/2
        self.handle_action(action_index)  # Применяем действие к птице
        self.apply_physics()  # Обновляем скорость и позицию под действием гравитации
        self.move_pipes()  # Сдвигаем трубы
        reward = self.config.alive_reward  # По умолчанию выдаём небольшую награду за выживание
        done = False  # Флаг окончания игры по умолчанию
        if self.is_collision():  # Проверяем столкновения после обновления состояний
            reward = self.config.collision_penalty  # Начисляем штраф за проигрыш
            done = True  # Завершаем игру
        else:  # Если столкновения не было
            for pipe in self.pipes:  # Проверяем, не пролетела ли птица очередную трубу
                if not pipe.scored and pipe.x + self.config.pipe_width < self.config.bird_x:  # Если труба позади и очки ещё не начислялись
                    pipe.scored = True  # Помечаем трубу как зачтённую
                    self.score += 1  # Увеличиваем счёт
                    reward = self.config.pipe_reward  # Выдаём дополнительную награду за прогресс
                    break  # Выходим из цикла, чтобы не начислять несколько раз за кадр
        self.frame_iteration += 1  # Увеличиваем счётчик кадров
        self.update_ui(action_index, game_number)  # Обновляем визуализацию и текстовую информацию
        self.clock.tick(self.config.fps)  # Ограничиваем FPS, чтобы симуляция шла с заданной скоростью
        return reward, done, self.score  # Возвращаем награду, флаг завершения и текущий счёт


In [ ]:

class LinearQNet(nn.Module):  # Нейросеть с одним скрытым слоем, аппроксимирующая функцию Q
    def __init__(self, input_size: int, hidden_size: int, output_size: int):  # Конструктор принимает размеры слоёв
        super().__init__()  # Вызываем инициализацию базового класса nn.Module
        self.linear1 = nn.Linear(input_size, hidden_size)  # Первый полносвязный слой: вход (11) → скрытый уровень (256)
        self.relu = nn.ReLU()  # Функция активации ReLU добавляет нелинейность
        self.linear2 = nn.Linear(hidden_size, output_size)  # Второй полносвязный слой: скрытый → выход (3 действий)

    def forward(self, x: torch.Tensor) -> torch.Tensor:  # Определяем прямое распространение сигнала
        x = self.linear1(x)  # Применяем первый линейный слой
        x = self.relu(x)  # Передаём результаты через ReLU
        x = self.linear2(x)  # Получаем оценки Q для каждого действия
        return x  # Возвращаем тензор с тремя значениями Q

class QTrainer:  # Класс, инкапсулирующий шаг оптимизации для Q-Learning
    def __init__(self, model: LinearQNet, lr: float, gamma: float):  # Принимаем модель, скорость обучения и гамму
        self.model = model  # Сохраняем ссылку на нейросеть
        self.lr = lr  # Запоминаем скорость обучения
        self.gamma = gamma  # Сохраняем коэффициент дисконтирования
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)  # Используем Adam для адаптивного подбора шагов
        self.criterion = nn.MSELoss()  # В качестве функции потерь применяем MSE между целевым и предсказанным Q

    def train_step(self, state, action, reward, next_state, done) -> None:  # Один шаг обучения по батчу или одиночному примеру
        state = torch.tensor(np.array(state), dtype=torch.float32)  # Преобразуем состояния в тензор с плавающей точкой
        next_state = torch.tensor(np.array(next_state), dtype=torch.float32)  # Аналогично обрабатываем следующее состояние
        action = torch.tensor(np.array(action), dtype=torch.float32)  # Конвертируем действия (one-hot) в тензор
        reward = torch.tensor(np.array(reward), dtype=torch.float32)  # Награды тоже превращаем в тензор
        if len(state.shape) == 1:  # Если на вход пришёл одиночный пример
            state = state.unsqueeze(0)  # Добавляем измерение батча
            next_state = next_state.unsqueeze(0)  # То же самое для следующего состояния
            action = action.unsqueeze(0)  # И для действия
            reward = reward.unsqueeze(0)  # А также для награды
            done = (done,)  # Превращаем булевое значение в кортеж для унификации обработки
        pred = self.model(state)  # Получаем предсказанные Q-значения для текущих состояний
        target = pred.clone()  # Создаём копию, которую будем менять, формируя обучающие цели
        for idx in range(len(done)):  # Проходим по всем элементам батча
            q_new = reward[idx]  # Базовое значение Q — это немедленная награда
            if not done[idx]:  # Если эпизод не завершён
                q_new = reward[idx] + self.gamma * torch.max(self.model(next_state[idx]))  # Добавляем дисконтированную будущую награду
            action_index = torch.argmax(action[idx]).item()  # Определяем, какое действие было совершено
            target[idx][action_index] = q_new  # Заменяем соответствующий элемент целевого Q новым значением
        self.optimizer.zero_grad()  # Обнуляем градиенты перед обратным проходом
        loss = self.criterion(pred, target)  # Вычисляем MSE между предсказанием и целями
        loss.backward()  # Считаем градиенты с помощью backpropagation
        self.optimizer.step()  # Обновляем веса модели

scores: List[int] = []  # Список для хранения очков каждой игры
mean_scores: List[float] = []  # Список средних значений очков по мере обучения

def plot_scores(scores: List[int], mean_scores: List[float]) -> None:  # Функция строит графики Score и Mean Score
    ipy_display.clear_output(wait=True)  # Очищаем предыдущее изображение графика
    ipy_display.display(plt.gcf())  # Показываем текущую фигуру Matplotlib
    plt.clf()  # Очищаем оси, чтобы перерисовать график с обновлёнными данными
    plt.title("История очков Flappy Bird")  # Заголовок графика
    plt.xlabel("Номер игры")  # Подпись оси X
    plt.ylabel("Score / Mean Score")  # Подпись оси Y
    plt.plot(scores, label="Score")  # Рисуем кривую очков каждой игры
    plt.plot(mean_scores, label="Mean Score")  # Рисуем кривую среднего значения
    plt.legend()  # Добавляем легенду для распознавания линий
    plt.grid(True)  # Включаем сетку для удобства чтения
    plt.pause(0.001)  # Даём Matplotlib время обновить рисунок

def create_ai_gif(model_path: str, config: GameConfig, gif_path: str, max_steps: int, duration: float = 0.06) -> None:  # Функция создаёт GIF с игрой обученного агента
    model = LinearQNet(11, 256, 3)  # Создаём такую же архитектуру, как при обучении
    model.load_state_dict(torch.load(model_path, map_location=torch.device("cpu")))  # Загружаем веса из сохранённого файла
    model.eval()  # Переводим модель в режим инференса
    game = FlappyBirdGame(config)  # Создаём новый экземпляр игры для записи
    frames: List[np.ndarray] = []  # Список кадров, которые войдут в GIF
    game.reset()  # Сбрасываем игру перед записью
    done = False  # Флаг завершения эпизода
    steps = 0  # Счётчик кадров очередного прогона
    while not done and steps < max_steps:  # Записываем, пока игра не окончена или не достигнут лимит кадров
        state = game.get_state()  # Получаем текущее состояние игры
        state_tensor = torch.tensor(state, dtype=torch.float32)  # Преобразуем его в тензор
        with torch.no_grad():  # Отключаем вычисление градиентов во время инференса
            prediction = model(state_tensor)  # Получаем оценки Q для доступных действий
        action_index = torch.argmax(prediction).item()  # Выбираем действие с максимальным Q
        final_move = [0, 0, 0]  # Заготавливаем вектор one-hot
        final_move[action_index] = 1  # Отмечаем выбранное действие как активное
        _, done, _ = game.play_step(final_move, game_number=0)  # Делаем шаг симуляции (номер игры 0, потому что это демонстрация)
        frame = pygame.surfarray.array3d(game.screen)  # Считываем поверхность окна pygame в виде массива
        frame = np.transpose(frame, (1, 0, 2))  # Транспонируем оси, чтобы получить привычный формат (H, W, C)
        frames.append(frame)  # Добавляем кадр в список
        steps += 1  # Увеличиваем счётчик шагов
    imageio.mimsave(gif_path, frames, duration=duration)  # Сохраняем собранные кадры в GIF с заданной скоростью


In [ ]:

class Agent:  # Класс, объединяющий игру, нейросеть, память и стратегию действий
    def __init__(self, config: GameConfig):  # Конструктор принимает конфигурацию
        self.config = config  # Сохраняем ссылку на конфигурацию
        self.n_games = 0  # Счётчик сыгранных игр
        self.epsilon = config.exploration_start  # Текущее значение epsilon
        self.gamma = config.gamma  # Коэффициент дисконтирования для удобного доступа
        self.memory: Deque[Experience] = deque(maxlen=config.max_memory)  # Создаём память опыта ограниченного размера
        self.model = LinearQNet(11, 256, 3)  # Инициализируем нейросеть согласно требованиям
        self.trainer = QTrainer(self.model, lr=config.lr, gamma=config.gamma)  # Создаём тренера с Adam и MSE
        self.game = FlappyBirdGame(config)  # Создаём экземпляр игры, с которым будет взаимодействовать агент
        self.best_score = 0  # Переменная для отслеживания максимального результата

    def remember(self, state, action, reward, next_state, done) -> None:  # Сохраняем опыт в память
        self.memory.append(Experience(state, action, reward, next_state, done))  # Добавляем пятиэлементный кортеж в deque

    def train_long_memory(self) -> None:  # Обучаемся на случайном батче из памяти опыта
        if len(self.memory) > self.config.batch_size:  # Если накоплено достаточно данных
            mini_sample = random.sample(self.memory, self.config.batch_size)  # Берём случайный батч фиксированного размера
        else:  # Если опыта пока мало
            mini_sample = list(self.memory)  # Берём все доступные элементы
        if mini_sample:  # Проверяем, что список не пуст
            states, actions, rewards, next_states, dones = zip(*mini_sample)  # Распаковываем батч по компонентам
            self.trainer.train_step(states, actions, rewards, next_states, dones)  # Выполняем шаг обучения на всём батче

    def train_short_memory(self, state, action, reward, next_state, done) -> None:  # Обучение на одном переходе
        self.trainer.train_step(state, action, reward, next_state, done)  # Передаём опыт напрямую тренеру

    def get_action(self, state: np.ndarray) -> List[int]:  # Функция выбирает действие с учётом epsilon-greedy
        self.epsilon = max(  # Корректируем epsilon с учётом нижней границы и скорости уменьшения
            self.config.exploration_end,  # Минимальная доля случайных действий, чтобы не застрять в локуме
            self.config.exploration_start - self.n_games * self.config.exploration_decay,  # Линейно уменьшаем epsilon после каждой игры
        )  # Получаем итоговое значение epsilon для текущей игры
        final_move = [0, 0, 0]  # Заготавливаем one-hot вектор действий
        if random.randint(0, 200) < self.epsilon:  # С вероятностью epsilon выбираем случайное действие
            move = random.randint(0, 2)  # Выбираем индекс действия случайным образом
            final_move[move] = 1  # Кодируем действие в формате one-hot
        else:  # Иначе полагаемся на нейросеть
            state0 = torch.tensor(state, dtype=torch.float32)  # Преобразуем состояние в тензор
            prediction = self.model(state0)  # Получаем оценки Q для каждого действия
            move = torch.argmax(prediction).item()  # Находим действие с максимальным Q
            final_move[move] = 1  # Записываем его в one-hot вектор
        return final_move  # Возвращаем выбранное действие


In [ ]:

config = GameConfig()  # Создаём объект конфигурации со всеми параметрами
agent = Agent(config)  # Создаём агента, передавая ему конфигурацию
scores = []  # Подготовим список для записи очков каждой игры
mean_scores = []  # Список для средних значений очков
total_score = 0  # Интегральная сумма очков для вычисления среднего
while agent.n_games < config.max_games:  # Запускаем обучение, пока не сыграем нужное количество игр
    old_state = agent.game.get_state()  # Снимаем состояние среды до выбора действия
    final_move = agent.get_action(old_state)  # Получаем действие от агента (с учётом epsilon-greedy)
    reward, done, score = agent.game.play_step(final_move, agent.n_games + 1)  # Делаем шаг в игре и получаем награду
    new_state = agent.game.get_state()  # Снимаем новое состояние после шага
    agent.train_short_memory(old_state, final_move, reward, new_state, done)  # Обновляем модель по одиночному опыту
    agent.remember(old_state, final_move, reward, new_state, done)  # Сохраняем переход в память Experience Replay
    if done:  # Если игра завершилась
        agent.game.reset()  # Сбрасываем игровое поле
        agent.n_games += 1  # Увеличиваем счётчик игр
        agent.train_long_memory()  # Делаем обучение на батче из памяти
        scores.append(score)  # Добавляем очки этой игры в историю
        total_score += score  # Обновляем суммарный счёт
        mean_score = total_score / agent.n_games  # Считаем среднее значение очков
        mean_scores.append(mean_score)  # Сохраняем среднее для графика
        if score > agent.best_score:  # Если достигнут новый личный рекорд
            agent.best_score = score  # Обновляем значение лучшего результата
            torch.save(agent.model.state_dict(), config.best_model_path)  # Сохраняем веса модели как лучшую версию
        plot_scores(scores, mean_scores)  # Обновляем графики Score/Mean Score
        print(f"Игра {agent.n_games}: Score={score} | Mean Score={mean_score:.2f} | Epsilon={agent.epsilon:.2f}")  # Логируем номер игры, очки и текущее epsilon одним сообщением


In [ ]:

create_ai_gif(  # Запускаем процедуру сохранения GIF с лучшим агентом
    model_path=config.best_model_path,  # Загружаем лучшую модель по итогам обучения
    config=config,  # Используем ту же конфигурацию игры, что и во время тренировки
    gif_path=config.gif_path,  # Сохраняем GIF в путь, определённый в конфигурации
    max_steps=config.gif_max_steps,  # Записываем ограниченное число кадров, чтобы ролик был компактным
    duration=0.06,  # Устанавливаем длительность кадра в секундах (чем меньше, тем быстрее GIF)
)  # Вызываем функцию создания GIF
print(f"GIF с игрой агента сохранён в файл: {config.gif_path}")  # Сообщаем пользователю путь к GIF


### Результаты

- Агент обучается минимум на 200 играх, что удовлетворяет требованию (>=150).
- Лучшая модель автоматически сохраняется в `best_flappy_q_agent.pth` и используется для генерации GIF.
- График `Score`/`Mean Score` помогает оценить динамику обучения прямо во время тренировки.
- Все параметры (learning rate, gamma, размеры слоёв, epsilon) подробно описаны в комментариях, а каждая строка кода снабжена пояснениями.
